# Vitessce Data Preparation Tutorial

# Export data to local files

## 1. Import dependencies

We need to import the classes and functions that we will be using from the corresponding packages.

In [ ]:
import os
import json
from urllib.parse import quote_plus
from os.path import join
from urllib.request import urlretrieve
from anndata import read_h5ad
import scanpy as sc

from vitessce import (
    VitessceWidget,
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
)

## 2. Download and process data

For this example, we need to download a dataset from the COVID-19 Cell Atlas https://www.covid19cellatlas.org/index.healthy.html#habib17.

In [ ]:
os.makedirs("data", exist_ok=True)
adata_filepath = join("data", "habib17.processed.h5ad")
urlretrieve('https://covid19.cog.sanger.ac.uk/habib17.processed.h5ad', adata_filepath)

adata.X = adata.X.tocsc()
top_dispersion = adata.var["dispersions_norm"][
    sorted(
        range(len(adata.var["dispersions_norm"])),
        key=lambda k: adata.var["dispersions_norm"][k],
    )[-51:][0]
]
adata.var["top_highly_variable"] = (
    adata.var["dispersions_norm"] > top_dispersion
)

## 3. Create the Vitessce configuration

Set up the configuration by adding the views and datasets of interest.

In [ ]:
vc = VitessceConfig(name='Habib et al', description='COVID-19 Healthy Donor Brain')
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(
        adata,
        mappings_obsm=["X_umap"],
        mappings_obsm_names=["UMAP"],
        cell_set_obs=["CellType"],
        cell_set_obs_names=["Cell Type"],
        expression_matrix="X",
        matrix_gene_var_filter="top_highly_variable"
))
scatterplot = vc.add_view(dataset, cm.SCATTERPLOT, mapping="X_umap")
cell_sets = vc.add_view(dataset, cm.CELL_SETS)
genes = vc.add_view(dataset, cm.GENES)
heatmap = vc.add_view(dataset, cm.HEATMAP)
vc.layout((scatterplot | (cell_sets / genes)) / heatmap);

## 4. Export files to a local directory

The `.export(to='files')` method on the view config instance will export files to the specified directory `out_dir`. The `base_url` parameter is required so that the file URLs in the view config point to the location where you ultimately intend to serve the files.

In [ ]:
config_dict = vc.export(to='files', base_url='http://localhost:3000', out_dir='./test')

## 5. Serve the files

Now that the files have been saved to the `./test` directory, they can be served by any static web server.

If you would like to serve the files locally, we recommend [http-server](https://github.com/http-party/http-server) which can be installed with NPM or Homebrew:
```sh
cd test
http-server ./ --cors -p 3000
```

## 6. View on vitessce.io

The returned view config dict can be converted to a URL, and if the files are served on the internet (rather than locally), this URL can be used to share the interactive visualizations with colleagues.

In [ ]:
vitessce_url = f"http://vitessce.io/?url=data:," + quote_plus(json.dumps(config_dict))
print(vitessce_url)